# Differents éléments d'un réseau de neurone

Nous avons vu les differentes fonctionnalités de base de theano précédemment, nous allons maintenant pouvoir nous interesser au différents blocs qui peuvent former un réseau de neuronne. Pour commencer nous avons vu un exemple de modèle de réseau de neuronne au tutoriel précédent. En général un réseau de neuronne sera composé d'un certain nombre de neuronne, ces neuronnes seront repartis en "layer" et ainsi chacun d'eux appliquera une opération mathématique sur ses entrées afin de générer sa sortie. Nous allons dans ce tutoriel voir les différents "layer" que nous pouvons utiliser dans un réseau de neuronnes:

## Softmax

Le premier type de "layer" que nous verrons est le softmax, il permet de transformer les valeurs des neurones en probabilité: $$\frac{valeur\_neuronne}{\sum_{k=0}^N valeur\_neuronne}$$  
Le Softmax sera toujours suivi d'une autre fonction : argmax, qui retournera l'indice du neuronne avec la plus haute probabilité. L'ensemble de ces deux fonctions est souvent utilisé en tant derniere couche de neurone afin de réaliser la prédiction.  
Afin d'utiliser la fonction softmax de theano nous avons besoin de définir des poids et des biais, pour cela nous les déclarons en tant que variables partagées.  
Commençons par importer les librairies nécessaire à ce tuto :

In [1]:
import numpy
import theano
import theano.tensor as T

### Softmax

Nous pouvons maintenant commencer à coder notre softmax :

In [2]:
input = 2

# On déclare les poids
W = theano.shared(
    value=numpy.zeros(
        (0, 10),
        dtype=theano.config.floatX
    ),
    name='W',
    borrow=True
)

# On déclare les biais
b = theano.shared(
    value=numpy.zeros(
        (10,),
        dtype=theano.config.floatX
    ),
    name='b',
    borrow=True
)

# On appele la fonction softmax
p_y_given_x = T.nnet.softmax(T.dot(input, W) + b)

# On stocke les poids et biais
params = [W, b]

### Argmax

Pour la fonction argmax nous n'avons besoin que de la sortie de la fonction softmax :

In [3]:
y_pred = T.argmax(p_y_given_x, axis=1)

## Les fonctions conv2d et pool

Nous allons maintenant voir un puissant outil des réseau de neurones: le layer convolutionnal.

### Conv2d

Il va fonctionner de la manière suivante: il applique un filtre sur une partie de la matrice d'entrée à la fois afin de créer une nouvelle matrice contenant plus d'information concentrée, le filtre sera dimensionné de sorte à récuperer les informations importantes.  
![Image 1](img/deep-learning-with-keras-48-638.jpg)

Pour utiliser la fonction conv2d nous avons besoin de définir plusieurs variables, la première sera la matrice d'entrée qui devra être redimensionnée pour correspondre à la taille d'entrée que nous indiquons à la fonction:

In [4]:
batch_size=600 # nombre de donnée en entrée
x = T.matrix('x')
input = x.reshape((batch_size, 1, 28, 28))
image_shape = (batch_size, 1, 28, 28)

Une fois l'entrée définit il nous faut définir un filtre pour la convolution ainsi que la taille de ce filtre, pour cela nous utiliserons une variable partagée:

In [5]:
nkerns = [20, 50] # définit la "profondeur" des filtres
poolsize=(2, 2)

filter_shape = (nkerns[0], 1, 5, 5)

fan_in = numpy.prod(filter_shape[1:])

fan_out = (filter_shape[0] * numpy.prod(filter_shape[2:]) //
           numpy.prod(poolsize))

W_bound = numpy.sqrt(6. / (fan_in + fan_out))

W = theano.shared(
    numpy.asarray(
        numpy.random.uniform(low=-W_bound, high=W_bound, size=filter_shape),
        dtype=theano.config.floatX
    ),
    borrow=True
)

Une fois toutes les variables définies nous pouvons appeler la fonction conv2d, ainsi que sauvegarder ses paramètres:

In [6]:
conv_out = T.nnet.conv2d(
    input=input,
    filters=W,
    filter_shape=filter_shape,
    input_shape=image_shape
)

params = [W]

### MaxPooling

La fonction maxpool va permettre de reduire la taille de la matrice sur laquelle on travaille, pour cela il nous faudra définir une poolSize qui sera la taille du kernel sur laquelle la fonction agira, la fonction prendra le max de chaque partie de la matrice afin de créer une nouvelle matrice de sortie. Exemple: avec une poolSize de (2,2) la taille de la matrice sera divisée par 2.  
Cette fonction est utilisée essentiellement après un conv layer pour récuperer les information les plus importantes.  
![image2](img/maxpool.jpeg)

In [7]:
from theano.tensor.signal import pool
pooled_out = pool.pool_2d(
    input=conv_out,
    ws=poolsize,
    ignore_border=True
)

## Les fonctions pour des réseaux dense

Il existe un certain nombre de fonction pour créer des réseaux dense dans la librairie theano.tensor.nnet, elle permette de créer des réseaux classique, mais peuvent aussi être utilisées dans des réseaux plus complexe pour rajouter une étape à un réseau, cela permet parfois de rendre le réseau plus puissant. Voici une liste des fonctions que l'on peut trouver dans dans cette librairie:


    Sigmoid
            sigmoid()
            ultra_fast_sigmoid()
            hard_sigmoid()

    Others
            softplus()
            softmax()
            softsign()
            relu()
            elu()
            selu()
            binary_crossentropy()
            sigmoid_binary_crossentropy()
            categorical_crossentropy()
            h_softmax()
            confusion_matrix

Nous avons déja vu la fonction Softmax dans ce tutoriel, toutes ces fonctions vont marcher selon plus ou moins le même principe que Softmax, vous pouvez trouver une aide pour chacune de ces fonctions sur le site officiel de [theano](http://deeplearning.net/software/theano/library/tensor/nnet/nnet.html)

Voyons ensemble une de ces fonctions qui est très utilisée dans les réseaux de neurones : la fonction relu.  
Pour cette fonction nous avons besoin de déclarer des poids et des biais comme pour la Softmax, pour cela nous utiliserons donc des variables partagées. Cette fonction va mettre à 0 tous les neurones à valeur négative et laisser les autres à leur valeur.  
![image 3](img/relu-activation-function-1.png)

In [8]:
input = 2

# On déclare les poids
W1 = theano.shared(
    value=numpy.zeros(
        (0, 10),
        dtype=theano.config.floatX
    ),
    name='W',
    borrow=True
)

# On déclare les biais
b1 = theano.shared(
    value=numpy.zeros(
        (10,),
        dtype=theano.config.floatX
    ),
    name='b',
    borrow=True
)

Puis nous faisons l'appel à notre fonction relu et nous enregistrons ses paramètres :

In [9]:
p_y_given_x = T.nnet.relu(T.dot(input, W1) + b1, alpha=0)

# On stocke les poids et biais
params = [W1, b1]

Il existe aussi une autre fonction dont nous n'avons pas encore parlé : la fonction tanh, elle va avoir le même effet que la fonction sigmoid mais entre l'intervalle [-1;1] alors que la sigmoid agit sur l'intervalle [0;1]

Nous avons vu quelque fonctions pour créer un réseau de neurones, il en existe beaucoup que je vous laisserai découvrir par vous même, vous avez maintenant toutes les cartes en main pour avancer. Il ne reste une derniere chose à voir : les patterns de layer convolutionnal. Il en existe un qui peut être utilisé :  
INPUT -> [[CONV -> RELU]\*N -> POOL?]\*M -> [FC -> RELU]\*K -> FC  
  
N,M,K sont des constantes qui indique le nombre de fois ou le layer est répété, il est possible d'avoir N,M,K = 0.  
? indique que l'on peut ou non mettre le layer  
CONV = layer convolutionnal  
RELU = layer relu  
POOL = layer maxpool  
FC = layer fully connected

Dans le prochain tutoriel nous mettrons tout cela en pratique pour créer un réseau de neurone répondant au problème de la détection de chiffre : le MNIST